In [43]:
import pandas as pd
import numpy as np

In [44]:
file = ('~/Downloads/purchase_data.csv')
pur_data = pd.read_csv(file)

In [45]:
player_count = len(pur_data['SN'].unique())
players_df = pd.DataFrame([{'Total Players': player_count}])
players_df.set_index('Total Players', inplace = True)
players_df

""
Total Players
576


In [46]:
no_dup_items = pur_data.drop_duplicates(['Item ID'], keep = 'last')
total_unique = len(no_dup_items)
total_pur = pur_data['Price'].count()
total_rev = round(pur_data['Price'].sum(),2)
avg_price = round(total_rev/total_pur, 2)
pur_analysis = pd.DataFrame([{
    
    "Number of Unique Items": total_unique,
    'Average Purchase Price': avg_price,
    'Total Purchases': total_pur,
    'Total Revenue': total_rev
}])
pur_analysis.style.format({'Average Purchase Price': '${:.2f}', 'Total Revenue': '${:,.2f}'})

,Average Purchase Price,Number of Unique Items,Total Purchases,Total Revenue
0,$3.05,183,780,"$2,379.77"


In [47]:
no_dup_players = pur_data.drop_duplicates(['SN'], keep ='last')
gender_counts = no_dup_players['Gender'].value_counts().reset_index()
gender_counts['% of Players'] = gender_counts['Gender']/player_count * 100
gender_counts.rename(columns = {'index': 'Gender', 'Gender': '# of Players'}, inplace = True)
gender_counts.set_index(['Gender'], inplace = True)
gender_counts.style.format({"% of Players": "{:.1f}%"})

,# of Players,% of Players
Gender,,
Male,484,84.0%
Female,81,14.1%
Other / Non-Disclosed,11,1.9%


In [48]:
pur_count_by_gen = pd.DataFrame(pur_data.groupby('Gender')['Gender'].count())
total_pur_by_gen = pd.DataFrame(pur_data.groupby('Gender')['Price'].sum())
pur_analysis_gen = pd.merge(pur_count_by_gen, total_pur_by_gen, left_index = True, right_index = True)
pur_analysis_gen.rename(columns = {'Gender': '# of Purchases', 'Price':'Total Purchase Value'}, inplace=True)
pur_analysis_gen['Average Purchase Price'] = pur_analysis_gen['Total Purchase Value']/pur_analysis_gen['# of Purchases']
pur_analysis_gen = pur_analysis_gen.merge(gender_counts, left_index = True, right_index = True)
pur_analysis_gen['Normalized Totals'] = pur_analysis_gen['Total Purchase Value']/pur_analysis_gen['# of Players']
pur_analysis_gen
del pur_analysis_gen['% of Players']
del pur_analysis_gen['# of Players']
pur_analysis_gen.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}', 'Normalized Totals': '${:.2f}'})

,# of Purchases,Total Purchase Value,Average Purchase Price,Normalized Totals
Gender,,,,
Female,113,$361.94,$3.20,$4.47
Male,652,$1967.64,$3.02,$4.07
Other / Non-Disclosed,15,$50.19,$3.35,$4.56


In [49]:
pur_data.loc[(pur_data['Age'] < 10), 'age_bin'] = "< 10"
pur_data.loc[(pur_data['Age'] >= 10) & (pur_data['Age'] <= 14), 'age_bin'] = "10 - 14"
pur_data.loc[(pur_data['Age'] >= 15) & (pur_data['Age'] <= 19), 'age_bin'] = "15 - 19"
pur_data.loc[(pur_data['Age'] >= 20) & (pur_data['Age'] <= 24), 'age_bin'] = "20 - 24"
pur_data.loc[(pur_data['Age'] >= 25) & (pur_data['Age'] <= 29), 'age_bin'] = "25 - 29"
pur_data.loc[(pur_data['Age'] >= 30) & (pur_data['Age'] <= 34), 'age_bin'] = "30 - 34"
pur_data.loc[(pur_data['Age'] >= 35) & (pur_data['Age'] <= 39), 'age_bin'] = "35 - 39"
pur_data.loc[(pur_data['Age'] >= 40), 'age_bin'] = "> 40"
pur_count_age = pd.DataFrame(pur_data.groupby('age_bin')['SN'].count())
avg_price_age = pd.DataFrame(pur_data.groupby('age_bin')['Price'].mean())
tot_pur_age = pd.DataFrame(pur_data.groupby('age_bin')['Price'].sum())
no_dup_age = pd.DataFrame(pur_data.drop_duplicates('SN', keep = 'last').groupby('age_bin')['SN'].count())
merge_age = pd.merge(pur_count_age, avg_price_age, left_index = True, right_index = True).merge(tot_pur_age, left_index = True, right_index = True).merge(no_dup_age, left_index = True, right_index = True)
merge_age.rename(columns = {"SN_x": "# of Purchases", "Price_x": "Average Purchase Price", "Price_y": "Total Purchase Value", "SN_y": "# of Purchasers"}, inplace = True)
merge_age['Normalized Totals'] = merge_age['Total Purchase Value']/merge_age['# of Purchasers']
merge_age.index.rename("Age", inplace = True)
merge_age.style.format({'Average Purchase Price': '${:.2f}', 'Total Purchase Value': '${:.2f}', 'Normalized Totals': '${:.2f}'})

,# of Purchases,Average Purchase Price,Total Purchase Value,# of Purchasers,Normalized Totals
Age,,,,,
10 - 14,28,$2.96,$82.78,22,$3.76
15 - 19,136,$3.04,$412.89,107,$3.86
20 - 24,365,$3.05,$1114.06,258,$4.32
25 - 29,101,$2.90,$293.00,77,$3.81
30 - 34,73,$2.93,$214.00,52,$4.12
35 - 39,41,$3.60,$147.67,31,$4.76
< 10,23,$3.35,$77.13,17,$4.54
> 40,13,$2.94,$38.24,12,$3.19


In [50]:
purchase_amt_by_SN = pd.DataFrame(pur_data.groupby('SN')['Price'].sum())
num_purchase_by_SN = pd.DataFrame(pur_data.groupby('SN')['Price'].count())
avg_purchase_by_SN = pd.DataFrame(pur_data.groupby('SN')['Price'].mean())
merged_top5 = pd.merge(purchase_amt_by_SN, num_purchase_by_SN, left_index = True, right_index = True).merge(avg_purchase_by_SN, left_index=True, right_index=True)
merged_top5.rename(columns = {'Price_x': 'Total Purchase Value', 'Price_y':'Purchase Count', 'Price':'Average Purchase Price'}, inplace = True)
merged_top5.sort_values('Total Purchase Value', ascending = False, inplace=True)
merged_top5 = merged_top5.head()
merged_top5.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}'})

,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Lisosia93,$18.96,5,$3.79
Idastidru52,$15.45,4,$3.86
Chamjask73,$13.83,3,$4.61
Iral74,$13.62,4,$3.40
Iskadarya95,$13.10,3,$4.37


In [60]:
top5_items_ID = pd.DataFrame(pur_data.groupby('Item ID')['Item ID'].count())
#top5_items_ID.sort_values(axis=4, ascending = False, inplace = True)
top5_items_ID = top5_items_ID.iloc[0:5][:]
top5_items_total = pd.DataFrame(pur_data.groupby('Item ID')['Price'].sum())
top5_items = pd.merge(top5_items_ID, top5_items_total, left_index = True, right_index = True)
no_dup_items = pur_data.drop_duplicates(['Item ID'], keep = 'last')
top5_merge_ID = pd.merge(top5_items, no_dup_items, left_index = True, right_on = 'Item ID')
top5_merge_ID = top5_merge_ID[['Item ID', 'Item Name', 'Item ID_x', 'Price_y', 'Price_x']]
top5_merge_ID.set_index(['Item ID'], inplace = True)
top5_merge_ID.rename(columns =  {'Item ID_x': 'Purchase Count', 'Price_y': 'Item Price', 'Price_x': 'Total Purchase Value'}, inplace=True)
top5_merge_ID.style.format({'Item Price': '${:.2f}', 'Total Purchase Value': '${:.2f}'})

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
0,Splinter,4,$1.28,$5.12
1,Crucifer,3,$3.26,$9.78
2,Verdict,6,$2.48,$14.88
3,Phantomlight,6,$2.49,$14.94
4,Bloodlord's Fetish,5,$1.70,$8.50


In [22]:
top5_profit = pd.DataFrame(pur_data.groupby('Item ID')['Price'].sum())
top5_profit.sort_values('Price', ascending = False, inplace = True)
top5_profit = top5_profit.iloc[0:5][:]
pur_count_profit = pd.DataFrame(pur_data.groupby('Item ID')['Item ID'].count())
top5_profit = pd.merge(top5_profit, pur_count_profit, left_index = True, right_index = True, how = 'left')
top5_merge_profit = pd.merge(top5_profit, no_dup_items, left_index = True, right_on = 'Item ID', how = 'left')
top5_merge_profit = top5_merge_profit[['Item ID', 'Item Name', 'Item ID_x', 'Price_y','Price_x']]
top5_merge_profit.set_index(['Item ID'], inplace=True)
top5_merge_profit.rename(columns = {'Item ID_x': 'Purchase Count', 'Price_y': 'Item Price', 'Price_x': 'Total Purchase Value'}, inplace = True)
top5_merge_profit.style.format({'Item Price': '${:.2f}', 'Total Purchase Value': '${:.2f}'})

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
